## CloudWatch Dashboard Setup

In [13]:
# Import Libraries
import boto3
import json
import pandas as pd
from time import strftime
import os

In [14]:
# Initialize CloudWatch Client
region = 'us-east-1'
endpoint_name = 'cardio-logistic-monitor-endpoint'
variant_name = 'AllTraffic'
dashboard_name = f"cardio_monitoring_dashboard_{strftime('%Y-%m-%d-%H-%M-%S')}"
bucket = 'sagemaker-us-east-1-531690656306'
s3_folder = 'cardio_project'
cloudwatch = boto3.client('cloudwatch', region_name=region)

In [19]:
# Define widgets for dashboard
widgets = [
    {
        "type": "metric",
        "x": 0, "y": 0, "width": 12, "height": 6,
        "properties": {
            "title": "CPU Utilization",
            "metrics": [
                ["AWS/SageMaker", "CPUUtilization", "EndpointName", endpoint_name, "VariantName", variant_name]
            ],
            "period": 300, "stat": "Average", "region": region
        }
    },
    {
        "type": "metric",
        "x": 12, "y": 0, "width": 12, "height": 6,
        "properties": {
            "title": "Memory Utilization",
            "metrics": [
                ["AWS/SageMaker", "MemoryUtilization", "EndpointName", endpoint_name, "VariantName", variant_name]
            ],
            "period": 300, "stat": "Average", "region": region
        }
    },
    {
        "type": "metric",
        "x": 0, "y": 6, "width": 12, "height": 6,
        "properties": {
            "title": "Disk Utilization",
            "metrics": [
                ["AWS/SageMaker", "DiskUtilization", "EndpointName", endpoint_name, "VariantName", variant_name]
            ],
            "period": 300, "stat": "Average", "region": region
        }
    },
    {
        "type": "metric",
        "x": 12, "y": 6, "width": 12, "height": 6,
        "properties": {
            "title": "Invocation Count",
            "metrics": [
                ["AWS/SageMaker", "Invocations", "EndpointName", endpoint_name, "VariantName", variant_name]
            ],
            "period": 300, "stat": "Sum", "region": region
        }
    },
    {
        "type": "metric",
        "x": 0, "y": 12, "width": 12, "height": 6,
        "properties": {
            "title": "Model Latency (ms)",
            "metrics": [
                ["AWS/SageMaker", "ModelLatency", "EndpointName", endpoint_name, "VariantName", variant_name]
            ],
            "period": 300, "stat": "Average", "region": region
        }
    },
    {
        "type": "metric",
        "x": 12, "y": 12, "width": 12, "height": 6,
        "properties": {
            "title": "Invocation Errors (5XX)",
            "metrics": [
                ["AWS/SageMaker", "Invocation5XXErrors", "EndpointName", endpoint_name, "VariantName", variant_name]
            ],
            "period": 300, "stat": "Sum", "region": region
        }
    }
]

# Deploy the dashboard
response = cloudwatch.put_dashboard(
    DashboardName=dashboard_name,
    DashboardBody=json.dumps({"widgets": widgets})
)

print(f"CloudWatch dashboard '{dashboard_name}' successfully created and deployed.")

CloudWatch dashboard 'cardio_monitoring_dashboard_2025-06-09-09-01-33' successfully created and deployed.


In [25]:
# Preview the widgets
widgets = [
    {
        "title": "CPU Utilization",
        "metrics": [["AWS/SageMaker", "CPUUtilization", "EndpointName", "cardio-logistic-monitor-endpoint", "VariantName", "AllTraffic"]],
        "unit": "%"
    },
    {
        "title": "Memory Utilization",
        "metrics": [["AWS/SageMaker", "MemoryUtilization", "EndpointName", "cardio-logistic-monitor-endpoint", "VariantName", "AllTraffic"]],
        "unit": "%"
    },
    {
        "title": "Disk Utilization",
        "metrics": [["AWS/SageMaker", "DiskUtilization", "EndpointName", "cardio-logistic-monitor-endpoint", "VariantName", "AllTraffic"]],
        "unit": "%"
    },
    {
        "title": "Invocation Count",
        "metrics": [["AWS/SageMaker", "Invocations", "EndpointName", "cardio-logistic-monitor-endpoint", "VariantName", "AllTraffic"]],
        "unit": "Count"
    },
    {
        "title": "Model Latency",
        "metrics": [["AWS/SageMaker", "ModelLatency", "EndpointName", "cardio-logistic-monitor-endpoint", "VariantName", "AllTraffic"]],
        "unit": "Milliseconds"
    },
    {
        "title": "Invocation 5XX Errors",
        "metrics": [["AWS/SageMaker", "Invocation5XXErrors", "EndpointName", "cardio-logistic-monitor-endpoint", "VariantName", "AllTraffic"]],
        "unit": "Count"
    }
]

# Display widget summary
for widget in widgets:
    print(f"\nWidget: {widget['title']}")
    print(f"   Metric: {widget['metrics'][0][1]}")
    print(f"   Unit: {widget['unit']}")


Widget: CPU Utilization
   Metric: CPUUtilization
   Unit: %

Widget: Memory Utilization
   Metric: MemoryUtilization
   Unit: %

Widget: Disk Utilization
   Metric: DiskUtilization
   Unit: %

Widget: Invocation Count
   Metric: Invocations
   Unit: Count

Widget: Model Latency
   Metric: ModelLatency
   Unit: Milliseconds

Widget: Invocation 5XX Errors
   Metric: Invocation5XXErrors
   Unit: Count


### Create .py file for Cloudwatch and save .ipynb to s3

In [26]:
# Write current dashboard creation code to file
py_filename = 'cloudwatch_dashboard_setup.py'

with open(py_filename, 'w') as f:
    f.write("# CloudWatch Dashboard Setup Code\n")
    f.write(f"region = '{region}'\n")
    f.write(f"endpoint_name = '{endpoint_name}'\n")
    f.write(f"variant_name = '{variant_name}'\n")
    f.write(f"dashboard_name = '{dashboard_name}'\n")
    f.write(f"bucket = '{bucket}'\n")
    f.write(f"s3_folder = '{s3_folder}'\n")
    f.write("# Full code omitted for brevity\n")

# Upload files to S3
s3_client = boto3.client('s3', region_name=region)

# Upload .ipynb
os.system(f"aws s3 cp cardio_model_monitoring.ipynb s3://{bucket}/{s3_folder}/cardio_cloudwatch.ipynb")

# Upload .py
s3_client.upload_file(py_filename, bucket, f"{s3_folder}/{py_filename}")

print("Both .ipynb and .py files successfully uploaded to S3.")

upload: ./cardio_model_monitoring.ipynb to s3://sagemaker-us-east-1-531690656306/cardio_project/cardio_cloudwatch.ipynb
Both .ipynb and .py files successfully uploaded to S3.


---------------

## Model and Data Reports

In [27]:
# Setup S3 client
s3 = boto3.client('s3', region_name='us-east-1')

# Define S3 bucket and baseline paths
bucket = 'sagemaker-us-east-1-531690656306'
baseline_prefix = 'cardio_data/baseline-results'

# Load statistics.json
statistics_obj = s3.get_object(Bucket=bucket, Key=f'{baseline_prefix}/statistics.json')
statistics_content = statistics_obj['Body'].read().decode('utf-8')
statistics = json.loads(statistics_content)

# Load constraints.json
constraints_obj = s3.get_object(Bucket=bucket, Key=f'{baseline_prefix}/constraints.json')
constraints_content = constraints_obj['Body'].read().decode('utf-8')
constraints = json.loads(constraints_content)

print("Statistics and constraints loaded successfully!")

Statistics and constraints loaded successfully!


In [28]:
print("\nBaseline Statistics Summary \n")

for feature in statistics.get('features', []):
    name = feature['name']
    stats = feature.get('numerical_statistics', {})
    
    if stats:  # Only for numerical features
        print(f"Feature: {name}")
        print(f" - Mean: {stats.get('mean', 'N/A')}")
        print(f" - Std Dev: {stats.get('standard_deviation', 'N/A')}")
        print(f" - Min: {stats.get('min', 'N/A')}")
        print(f" - Max: {stats.get('max', 'N/A')}")
        print("----------------------------------------------------")
    else:
        print(f"Feature: {name} (non-numeric or no statistics)")


Baseline Statistics Summary 

Feature: age
 - Mean: 52.83139577392703
 - Std Dev: N/A
 - Min: 29.0
 - Max: 64.0
----------------------------------------------------
Feature: gender
 - Mean: 1.3487753162243181
 - Std Dev: N/A
 - Min: 1.0
 - Max: 2.0
----------------------------------------------------
Feature: height_ft
 - Mean: 5.393331139869341
 - Std Dev: N/A
 - Min: 3.28
 - Max: 6.79
----------------------------------------------------
Feature: weight_lbs
 - Mean: 163.42664692552245
 - Std Dev: N/A
 - Min: 66.14
 - Max: 440.92
----------------------------------------------------
Feature: systolic_bp
 - Mean: 126.64601886378591
 - Std Dev: N/A
 - Min: 90.0
 - Max: 200.0
----------------------------------------------------
Feature: diastolic_bp
 - Mean: 81.32382832492506
 - Std Dev: N/A
 - Min: 60.0
 - Max: 120.0
----------------------------------------------------
Feature: cholesterol
 - Mean: 1.3643635300138919
 - Std Dev: N/A
 - Min: 1.0
 - Max: 3.0
-------------------------------

In [29]:
print("\nBaseline Constraints Summary \n")

for feature in constraints.get('features', []):
    name = feature['name']
    violations = feature.get('violations', [])
    
    if violations:
        print(f"Feature '{name}' has violations: {violations}")
    else:
        print(f"Feature '{name}' has no violations detected.")


Baseline Constraints Summary 

Feature 'age' has no violations detected.
Feature 'gender' has no violations detected.
Feature 'height_ft' has no violations detected.
Feature 'weight_lbs' has no violations detected.
Feature 'systolic_bp' has no violations detected.
Feature 'diastolic_bp' has no violations detected.
Feature 'cholesterol' has no violations detected.
Feature 'gluc' has no violations detected.
Feature 'smoke' has no violations detected.
Feature 'alco' has no violations detected.
Feature 'active' has no violations detected.
Feature 'cardio' has no violations detected.
Feature 'bmi' has no violations detected.
Feature 'age_group' has no violations detected.
Feature 'cholesterol_label' has no violations detected.
Feature 'pulse_pressure' has no violations detected.
Feature 'chol_bmi_ratio' has no violations detected.
Feature 'height_in' has no violations detected.
Feature 'age_years' has no violations detected.
Feature 'is_hypertensive' has no violations detected.
Feature 'bp

In [30]:
# Save Notebook to S3
!aws s3 cp cardio_cloudwatch_and_data_reports.ipynb s3://sagemaker-us-east-1-531690656306/cardio_project/cardio_cloudwatch_and_data_reports.ipynb

upload: ./cardio_cloudwatch_and_data_reports.ipynb to s3://sagemaker-us-east-1-531690656306/cardio_project/cardio_cloudwatch_and_data_reports.ipynb
